In [1]:
import pandas as pd
import re
from collections import defaultdict

In [2]:
income_file_path = "resources/state_income_data.csv"

In [3]:
income_df = pd.read_csv(income_file_path)

In [4]:
income_df.head()

,State,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
0,United States,"68,703","63,179","61,372","59,039","56,516","53,657","53,585","51,017","50,054","49,276"
1,Alabama,"56,200","49,936","51,113","47,221","44,509","42,278","47,320","43,464","42,590","40,933"
2,Alaska,"78,394","68,734","72,231","75,723","75,112","67,629","72,472","63,648","57,431","57,848"
3,Arizona,"70,674","62,283","61,125","57,100","52,248","49,254","52,611","47,044","48,621","46,896"
4,Arkansas,"54,539","49,781","48,829","45,907","42,798","44,922","39,376","39,018","41,302","38,587"


In [5]:
income_df.columns

Index(['State', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012',
       '2011', '2010'],
      dtype='object')

In [6]:
income_df.rename({"State": "state"}, 
          axis = "columns", inplace = True)

In [7]:
income_df.head()

,state,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
0,United States,"68,703","63,179","61,372","59,039","56,516","53,657","53,585","51,017","50,054","49,276"
1,Alabama,"56,200","49,936","51,113","47,221","44,509","42,278","47,320","43,464","42,590","40,933"
2,Alaska,"78,394","68,734","72,231","75,723","75,112","67,629","72,472","63,648","57,431","57,848"
3,Arizona,"70,674","62,283","61,125","57,100","52,248","49,254","52,611","47,044","48,621","46,896"
4,Arkansas,"54,539","49,781","48,829","45,907","42,798","44,922","39,376","39,018","41,302","38,587"


In [8]:
def condense_date_columns(df):
    # Grab non-date columns from dataframe
    non_date_columns = [x for x in df.columns if x[0].isalpha()]
    
    # Create data and non-date dataframes based on original df
    non_date_df = df[non_date_columns]
    date_df = df.drop(columns=non_date_columns)
    
    # Initialize final data dictionary
    data = defaultdict(list)
    
    # Loop through dates
    for col_date in date_df.columns:
        
        # Loop through values for each date
        for i, x in enumerate(list(date_df[col_date])):
            
            # Loop through non-date values
            for col_non_date in non_date_df.columns:
                
                # Append non-date value to non-date column
                data[str(col_non_date)].append(non_date_df[col_non_date][i])
            
            # Append date and price values
            data["date"].append(str(col_date))
            data["income"].append(float(x.replace(",","")))
            
    return pd.DataFrame(data)

In [9]:
final_income_df = condense_date_columns(income_df)

In [10]:
print(final_income_df.shape)
final_income_df.head()

(520, 3)


,state,date,income
0,United States,2019,68703.0
1,Alabama,2019,56200.0
2,Alaska,2019,78394.0
3,Arizona,2019,70674.0
4,Arkansas,2019,54539.0


In [11]:
# Create primary key column derived from unique region_id and date
final_income_df['region_date'] = final_income_df.apply(lambda row: row.state.lower().replace(" ","") + row.date.replace("-",""), axis=1)

In [12]:
final_income_df.columns

Index(['state', 'date', 'income', 'region_date'], dtype='object')

In [13]:
reordered_columns = ['region_date', 'state', 'date', 'income']

In [14]:
final_income_df = final_income_df[reordered_columns]
final_income_df.head()

,region_date,state,date,income
0,unitedstates2019,United States,2019,68703.0
1,alabama2019,Alabama,2019,56200.0
2,alaska2019,Alaska,2019,78394.0
3,arizona2019,Arizona,2019,70674.0
4,arkansas2019,Arkansas,2019,54539.0


In [15]:
final_income_df.to_csv("resources/final_income_data.csv", index=False)